# 🔰PyTorchでニューラルネットワーク基礎 #13 【音楽分類01・conv1d】

* ネットワークの構造の理解のためにデータサイズを小さくして、色んなパターンを試したい！

## 内容
* Qiitaの記事と連動しています。
* 1次元畳み込みとプーリング層を利用して音楽ジャンル分類を行う。

## 注意
* [Kaggle: GTZAN Dataset - Music Genre Classification](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification)からGTZANデータセットをダウンロードする必要があります。
* **データセットtrain_data.npzを作成しないと動作しないので注意してください**



## 目的
1. nn.Sequential()を使ったネットワークの記述方法
2. batchnorm1dの有無
3. adaptiveavgpool1dの有用性


## 音声分類
* 1次元畳み込みとbatchnorm1dを利用して音声分類を行う。
* train_data.npz : 各4ファイル、5秒、4分類
### データについて
* sampling rate 22050
* 系列長の時間：5秒 (22050*5)

### テスト精度
* 重複なし：90〜100%


In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
data = np.load("./data/train_data.npz")
x = data["x"]
t = data["t"]
x.shape, t.shape

((96, 110250), (96,))

# 1次元畳み込み層と1次元プーリング層

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

X = torch.FloatTensor(x).to(device).view(x.shape[0], 1, x.shape[1])
T = torch.LongTensor(t).to(device)

x_train, x_test, t_train, t_test = train_test_split(X, T, test_size=0.2, stratify=t, random_state=55)
x_train.shape, x_test.shape, t_train.shape, t_test.shape, t_test

device: cuda


(torch.Size([76, 1, 110250]),
 torch.Size([20, 1, 110250]),
 torch.Size([76]),
 torch.Size([20]),
 tensor([0, 1, 3, 0, 0, 2, 1, 1, 0, 2, 3, 2, 2, 1, 0, 1, 3, 3, 2, 3],
        device='cuda:0'))

## ネットワーク構造

In [4]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()        
        self.features = nn.Sequential(
            # 第1ブロック
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=500, stride=16),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2),
            
            # 第2ブロック
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=100, stride=16),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2),
            
            # 第3ブロック
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=10, stride=4),
            nn.BatchNorm1d(num_features=256),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool1d(output_size=8)  # 固定サイズの出力
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=256 * 8, out_features=512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=128),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=4)
        )
        
    def forward(self, x):
        h = self.features(x)
        y = self.classifier(h)
        return y

In [5]:
model = DNN()
model.to(device)

DNN(
  (features): Sequential(
    (0): Conv1d(1, 64, kernel_size=(500,), stride=(16,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 128, kernel_size=(100,), stride=(16,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(128, 256, kernel_size=(10,), stride=(4,))
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): AdaptiveAvgPool1d(output_size=8)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=512, bias=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()

In [6]:
# 損失関数と最適化関数の定
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)

In [7]:
def accuracy(y, t):
    _,argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy


In [8]:
LOOP = 100
for epoch in range(LOOP):
    model.train()
    optimizer.zero_grad()  
    y = model(x_train)
    loss = criterion(y, t_train)
    acc  = accuracy(y, t_train)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0:
        print(f"{epoch}: loss: {loss.item()},\tacc:{acc}")   # 損失と精度の表示

19: loss: 0.1402178853750229,	acc:0.9868421052631579
39: loss: 0.0501951202750206,	acc:1.0
59: loss: 0.030047224834561348,	acc:1.0
79: loss: 0.0158524289727211,	acc:1.0
99: loss: 0.016375450417399406,	acc:1.0


# テスト用データを使って検証

In [9]:
model.eval()
with torch.inference_mode():
    y_test = model(x_test)
test_acc = accuracy(y_test, t_test)
test_acc

0.95